<a href="https://colab.research.google.com/github/sznajdr/sumpdf/blob/main/longoform_doc_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# example - long-t5-tglobal-base-16384-book-summary

Link to model card: [pszemraj/long-t5-tglobal-base-16384-book-summary](https://huggingface.co/pszemraj/long-t5-tglobal-base-16384-book-summary)


by [Peter](https://github.com/pszemraj)


---


In [ ]:
%%capture
#@markdown set up auto-formatting of cells in notebook

from IPython.display import HTML, display


def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )
get_ipython().events.register("pre_run_cell", set_css)

In [ ]:
#@markdown GPU stats
!nvidia-smi

Sat Mar 18 12:02:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    31W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U -q transformers

In [ ]:
#@markdown utils
from transformers.utils.logging import set_verbosity

set_verbosity(40)

import warnings
# ignore hf pipeline complaints
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')
warnings.filterwarnings("ignore", category=FutureWarning, module='transformers')

In [ ]:
example_url = "https://pastebin.com/MJRSa9tY" #@param {type:"string"}
filename = 'example.txt'
!wget -O $filename $example_url

load example

In [ ]:
#@markdown read in text
import pprint as pp
with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
    long_text = f.read()

print(long_text[:1500])
print("\n" * 3,f"plus an additional {len(long_text)-1500} chars..")

load model.


In [ ]:
#@markdown load model
import torch
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    "pszemraj/long-t5-tglobal-base-16384-book-summary",
    device=0 if torch.cuda.is_available() else -1,
)


summarize!


In [ ]:
#@markdown summarization

params = {
    "max_length": 8000,
    "min_length": 300,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "repetition_penalty": 3.5,
    "length_penalty": 0.3,
    "encoder_no_repeat_ngram_size": 3,
    "num_beams": 4,
} # parameters for text generation out of model


In [ ]:
#@markdown return output

%%time
result = summarizer(long_text, **params)

print(result[0]['summary_text'])